## FPL GAFFER TOOL CALLING
This notebook contains experiments with boilerplate tools, showing how FPL Gaffer would chose the tools to call based on prompts from a user during conversations.

In [83]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import os
import asyncio
import json
from typing import List, Literal, Callable, Any
from pydantic import BaseModel, Field, ConfigDict
from langchain.tools import Tool, BaseTool
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from IPython.display import display, Image

In [2]:
# TODO: Use boilerplate functions for tools, assess
# with an LLM to test the tools that would be called
# for different kinds of prompts.

# NOTE: Use proper documentation for the notebook.

In [7]:
class AsyncFPLTool(BaseTool):
    """Custom async tool wrapper to handle async execution in LangGraph context."""
    func: Callable
    input_schema: BaseModel = None

    # Allow custom types
    model_config = ConfigDict(arbitrary_types_allowed=True)

    async def _arun(self, **kwargs) -> Any:
        """Async execution of the tool."""
        try:
            return await self.func(**kwargs)
        except Exception as e:
            return {"error": e}

    def _run(self, **kwargs) -> Any:
        """Fallback sync wrapper."""
        try:
            return asyncio.run(self.func(**kwargs))
        except Exception as e:
            return {"error": e}

# Tool input schemas
class NewsSearchInput(BaseModel):
    """Input schema for the news search tool."""
    query: str = Field(..., description="The search query for news search.")

class PlayerByPositionInput(BaseModel):
    """Input schema for the player by position tool."""
    position: Literal["GKP", "DEF", "MID", "FWD"] = Field(..., description="Position to search for. One of: GK, DEF, MID, FWD.")
    max_price: float = Field(15.0, description="Maximum player price to search for in millions.")


class PlayerDataInput(BaseModel):
    """Input schema for the player data tool."""
    player_names: List[str] = Field(..., description="List of player(s) names to fetch data for.")


class FixturesForRangeInput(BaseModel):
    """Input schema for the fixtures for range tool."""
    num_gameweeks: int = Field(..., description="Number of upcoming gameweeks to fetch fixtures for.")

class UserTeamInfoInput(BaseModel):
    """Input schema for the user team info tool."""
    manager_id: int = Field(..., description="The user's FPL manager ID.")
    gameweek: int = Field(..., description="The current gameweek number to fetch data for.")

/home/d3ju/Documents/fpl-gaffer/.venv/lib64/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "input_schema" in "AsyncFPLTool" shadows an attribute in parent "BaseTool"
  warnings.warn(


In [32]:
# Boilerplate tool functions
def news_search_tool(query: str):
    print(f"[news_search_tool] query: {query}")

def get_user_team_info_tool(manager_id: int, gameweek: int):
    print(f"[get_user_team_info_tool] manager_id={manager_id}, gameweek={gameweek}")

def get_players_by_position_tool(position: Literal["GKP", "DEF", "MID", "FWD"], max_price: float):
    print(f"[get_players_by_position_tool] position={position}, max_price={max_price}")

def get_player_data_tool(player_names: List[str]):
    print(f"[get_player_data_tool] player_names={player_names}")

def get_fixtures_for_range_tool(num_gameweeks: int):
    print(f"[get_fixtures_for_range_tool] num_gameweeks={num_gameweeks}")

# Create tool list
tools = [
        AsyncFPLTool(
            name="news_search_tool",
            description="Search for FPL news, expert analysis, injury updates, press conference information, etc."
                        "Use this when you need information about player/team news, injury, expert opinions, or "
                        "general FPL updates.",
            func=news_search_tool,
            args_schema=NewsSearchInput
        ),
        AsyncFPLTool(
            name="get_user_team_info_tool",
            description="Get comprehensive information about a user's FPL team including squad, transfers, "
                        "and finances. Use this when you need information about the user's team, players, or "
                        "financial situation.",
            func=get_user_team_info_tool,
            args_schema=UserTeamInfoInput
        ),
        AsyncFPLTool(
            name="get_players_by_position_tool",
            description="Get players by position and max price. Use this when you need information for player "
                        "replacements or transfer suggestions based on position and budget.",
            func=get_players_by_position_tool,
            args_schema=PlayerByPositionInput
        ),
        AsyncFPLTool(
            name="get_player_data_tool",
            description="Get detailed player data including stats, form, and injuries. Use this when you need "
                        "information about specific players.",
            func=get_player_data_tool,
            args_schema=PlayerDataInput
        ),
        AsyncFPLTool(
            name="get_fixtures_for_range_tool",
            description="Get fixtures from the current gameweek to the next x gameweeks. Use this when you need "
                        "information about upcoming fixtures or planning for future gameweeks.",
            func=get_fixtures_for_range_tool,
            args_schema=FixturesForRangeInput
        )
    ]

TOOLS_DESCRIPTION = "\n".join(
    f"{t.name}: {t.description}" for t in tools
)

In [76]:
MESSAGE_ANALYSIS_PROMPT_TEMPLATE = """
You are an FPL conversational assistant that needs to decide the tools to call to assist the user's query.


Available tools:
{tools}

Determine which tools to call and in what order. Do NOT provide any extra text, and do not add tools
that are not listed above.

Ouput MUST be one of:
1. For a single tool call, a List of the tool name like so:
["<tool_name>"]

2. For multiple tool calls, a LIST of JSON objects with tool names and arguments
like so: [{{ "name": "<name>" }}, {{ "name": "<name>" }}]

3.If no tool fits, respond with:
{{"name": "none", "message": "<natural language reply>" }}
"""

FPL_GAFFER_SYSTEM_PROMPT = """
You are FPL Gaffer, an FPL co-manager that helps user's to make FPL decisions and gives good
data driven suggestions.

Provide response to the user's query, and only answer based on factual data. Do NOT give responses
that you are have not gotten through facts.
"""

prompt = PromptTemplate.from_template(MESSAGE_ANALYSIS_PROMPT_TEMPLATE)
prompt = prompt.format(tools=tools)

print(prompt)


You are an FPL conversational assistant that needs to decide the tools to call to assist the user's query.


Available tools:
[AsyncFPLTool(name='news_search_tool', description='Search for FPL news, expert analysis, injury updates, press conference information, etc.Use this when you need information about player/team news, injury, expert opinions, or general FPL updates.', args_schema=<class '__main__.NewsSearchInput'>, func=<function news_search_tool at 0x7f0c48ecefc0>), AsyncFPLTool(name='get_user_team_info_tool', description="Get comprehensive information about a user's FPL team including squad, transfers, and finances. Use this when you need information about the user's team, players, or financial situation.", args_schema=<class '__main__.UserTeamInfoInput'>, func=<function get_user_team_info_tool at 0x7f0c48ecf060>), AsyncFPLTool(name='get_players_by_position_tool', description='Get players by position and max price. Use this when you need information for player replacements or t

In [ ]:
# TODO: Test running the tools directly to make this a single node
# instead of tool selection and execution nodes.

In [77]:
# Initialize llm
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.3-70b-versatile",
    temperature=0.4
).bind_tools(tools)

In [91]:
# def run_query(q: str):
#     messages = [FPL_GAFFER_SYSTEM_PROMPT, HumanMessage(content=q)]
#     result = llm.invoke(messages)
#     return result

# from langchain.schema import HumanMessage, ToolMessage

def run_query(q: str):
    # 1️⃣ Send user question
    messages = [SystemMessage(content=FPL_GAFFER_SYSTEM_PROMPT),
                HumanMessage(content=q)]
    step1 = llm.invoke(messages)
    print(step1)

    # 2️⃣ Check if the model requested a tool
    if step1.tool_calls:
        for call in step1.tool_calls:
            if call["name"] == "news_search_tool":
                # run your actual tool function
                result = news_search_tool(**call["args"])

                # 3️⃣ Send the tool’s result back
                messages.append(step1)  # the model’s tool-call message
                messages.append(ToolMessage(
                    content=result,
                    name=call["name"],
                    tool_call_id=call["id"],
                ))

                # 4️⃣ Ask model for the final answer
                step2 = llm.invoke(messages)
                return step2.content

    # If no tools, it already gave the answer
    return step1.content


In [92]:
run_query("Is Salah injured?")

content='' additional_kwargs={'tool_calls': [{'id': 'bgrwdbj6p', 'function': {'arguments': '{"query":"Mohamed Salah injury update"}', 'name': 'news_search_tool'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 889, 'total_tokens': 909, 'completion_time': 0.065575434, 'prompt_time': 0.072307931, 'queue_time': 0.085868543, 'total_time': 0.137883365}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--181dfbb2-96e0-487c-8bee-39c8e03d8c96-0' tool_calls=[{'name': 'news_search_tool', 'args': {'query': 'Mohamed Salah injury update'}, 'id': 'bgrwdbj6p', 'type': 'tool_call'}] usage_metadata={'input_tokens': 889, 'output_tokens': 20, 'total_tokens': 909}
[news_search_tool] query: Mohamed Salah injury update


''

In [93]:
response = run_query("Who should I captain this gameweek?")

content='' additional_kwargs={'tool_calls': [{'id': '6ky51y1a4', 'function': {'arguments': '{"query":"FPL captain picks gameweek"}', 'name': 'news_search_tool'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 894, 'total_tokens': 916, 'completion_time': 0.070730876, 'prompt_time': 0.073794497, 'queue_time': 0.083312617, 'total_time': 0.144525373}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--3a44779b-dea6-4059-8ce8-568598ead758-0' tool_calls=[{'name': 'news_search_tool', 'args': {'query': 'FPL captain picks gameweek'}, 'id': '6ky51y1a4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 894, 'output_tokens': 22, 'total_tokens': 916}
[news_search_tool] query: FPL captain picks gameweek


In [87]:
print(response)

In [88]:
run_query("Do you suggest I change any player from my team?")

''

In [94]:
response = run_query("Based on fixture difficuly in the next 5 gameweeks, what players do you think I should sell, and what teams can I invest in?")

content='' additional_kwargs={'tool_calls': [{'id': '8fzh692t9', 'function': {'arguments': '{"num_gameweeks":5}', 'name': 'get_fixtures_for_range_tool'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 916, 'total_tokens': 937, 'completion_time': 0.053313133, 'prompt_time': 0.073948585, 'queue_time': 0.084808599, 'total_time': 0.127261718}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--0b8ee56f-aefb-4919-8f29-d07792be8586-0' tool_calls=[{'name': 'get_fixtures_for_range_tool', 'args': {'num_gameweeks': 5}, 'id': '8fzh692t9', 'type': 'tool_call'}] usage_metadata={'input_tokens': 916, 'output_tokens': 21, 'total_tokens': 937}


In [90]:
response

''